<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/C3TR_Adapter_hqq_v2_Paid_Colab_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日英・英日翻訳サンプルコード(Japanese-English English-Japanese Translation sample code)


C3TR-Adapter Version2 hqq量子化版を使って日本語を英語、英語を翻訳するサンプルコードです。  
Sample code to translate Japanese to English and English to Japanese using C3TR-Adapter Version2 hqq quantized version.

残念ながら無料版Colabでは動きません。L4かA100を指定してください  
Unfortunately, it doesn't work with the free version of Colab. Please specify L4 or A100.  

上段メニューの「ランタイム」→「すべてのセルを実行」で実行してください  
Please execute it by clicking "Runtime" -> "Execute All Cells" in the upper menu  

もしくは各セルの[ ]の部分をクリックして実行してください  
Or click on the [ ] part of each cell to execute it.  


In [1]:
%%bash
git clone https://github.com/mobiusml/hqq
cd hqq
pip install .

Processing /content/hqq
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 13.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nv

Cloning into 'hqq'...


In [2]:
!pip install transformers==4.41.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 71.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0


## 上段メニューより「ランタイム」→「セッションを再起動」してください
## From the upper menu, click "runtime" -> "Restart Session".

In [3]:
import torch, os
from hqq.engine.hf import AutoTokenizer
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.models.hf.base import AutoHQQHFModel

model_id = "webbigdata/C3TR-Adapter_hqq"
os.environ["TOKENIZERS_PARALLELISM"]  = "1"
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32       = True

compute_dtype = torch.bfloat16
model     = AutoHQQHFModel.from_quantized(model_id, compute_dtype=compute_dtype)
tokenizer = AutoTokenizer.from_pretrained(model_id)

patch_linearlayers(model, patch_add_quant_config,
                          BaseQuantizeConfig(nbits=4, group_size=64, quant_scale=False, quant_zero=False, axis=1))
HQQLinear.set_backend(HQQBackend.PYTORCH)
#model.eval();

from hqq.utils.patching import prepare_for_inference
prepare_for_inference(model, backend="torchao_int4")


prompt_text = """You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

### Instruction:
Translate Japanese to English.
When translating, please use the following hints:
[writeing_style: web-fiction]
[リュグナー: Lügner]
[フリーレン: Frieren]
[フリーレン_first_person_and_ending: 私, だね, よね]
[フェルン: Fern]
[フェルン_first_person_and_ending: 私, です, ございます]
[ゾルトラーク: Soul track]

### Input:
リュグナー「あの小娘の所作には面影があった。私は昔、同じ魔法を受けたことがある。・・・そうか、思い出した。フリーレンだ。人類のゾルトラークの研究解析に大きく貢献し、歴史上で最も多くの魔族を葬り去った魔法使い。葬送のフリーレン。私の嫌いな天才だ。」

・・・

フリーレン：魔法は好き？
フェルン：ほどほどでございます。
フリーレン：私と同じだ。

...

フェルン：フリーレン様は本当に魔法がお好きなのですね。
フリーレン：ほどほどだよ。フェルンと同じで。
フェルン：少し違うような気がします。
フリーレン：同じだよ。
### Response:
"""

tokens = tokenizer(prompt_text, return_tensors="pt",
        padding=True, max_length=1600, truncation=True).to("cuda:0").input_ids

output = model.generate(
        input_ids=tokens,
        max_new_tokens=800,
        do_sample=True,
        num_beams=3, temperature=0.5, top_p=0.3,
        repetition_penalty=1.0)
print(tokenizer.decode(output[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
100%|██████████| 197/197 [00:00<00:00, 6543.89it/s]


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

<bos>You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th

In [13]:

prompt_text = """You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

### Instruction:
Translate English to Japanese.
When translating, please use the following hints:
[writeing_style: casual]
[リュグナー: Lügner]
[フリーレン: Frieren]
[フリーレン_character_style: female, casual]
[フリーレン_first_person_and_ending: 私, だよ, だね]
[フェルン: Fern]
[フェルン_character_style: female, humility]
[フェルン_first_person_and_ending: 私, ございます]
[Soul track: ゾルトラーク]

### Input:
Lügner: "There was something familiar in that girl's mannerisms. I once received the same magic myself. ... Oh, I remember now. Frieren. The magician who contributed greatly to the study and analysis of humanity's Soul Track and laid to rest the most number of demons in history. Funeral Frieren. My least favorite genius."

...

Frieren: Do you like magic?
Fern: Not too much.
Frieren: Same here.

...

Fern: Lady Frieren really likes magic, doesn't she?
Frieren: Somewhat. Same as Fern.
Fern: Seems like there's a slight difference.
Frieren: Same here.
### Response:
"""

tokens = tokenizer(prompt_text, return_tensors="pt",
        padding=True, max_length=1600, truncation=True).to("cuda:0").input_ids

output = model.generate(
        input_ids=tokens,
        max_new_tokens=800,
        do_sample=True,
        num_beams=3, temperature=0.5, top_p=0.3,
        repetition_penalty=1.0)
print(tokenizer.decode(output[0]))

<bos>You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th